# As supervised problem

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


symbolic link created for D:\Anaconda3\lib\site-packages\spacy\data\en <<===>> D:\Anaconda3\lib\site-packages\en_core_web_sm

    Linking successful
    D:\Anaconda3\lib\site-packages\en_core_web_sm -->
    D:\Anaconda3\lib\site-packages\spacy\data\en

    You can now load the model via spacy.load('en')



In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,responsible,previous,waiting,sixteen,naturally,little,teach,fact,bottle,correct,...,youthful,perfectly,negative,impossible,artificial,sink,glad,liberality,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\Users\Mike\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.992

Test set score: 0.8083832335329342


## BoW with Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\Mike\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(250, 1647) (250,)
Training set score: 0.976

Test set score: 0.8862275449101796


## BoW with Gradient Boosting

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.968

Test set score: 0.8143712574850299


### Same model, new inputs

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6330275229357798


col_0,Austen,Carroll
row_0,,
Austen,120,20
Carroll,60,18


# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%

In [15]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

C:\Users\Mike\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [16]:
print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.688

Test set score: 0.6826347305389222


In [17]:
alice_sent_len = [[len(sent), 'Carroll'] for sent in alice_doc.sents]
persuasion_sent_len = [[len(sent), 'Austen'] for sent in persuasion_doc.sents]

In [18]:
sent_len = pd.DataFrame(alice_sent_len + persuasion_sent_len)
sent_len.head(5)

,0,1
0,67,Carroll
1,63,Carroll
2,33,Carroll
3,3,Carroll
4,6,Carroll


In [19]:
sent_len.describe()

,0
count,417.000000
mean,31.117506
std,26.967539
min,1.000000
25%,12.000000
50%,23.000000
75%,42.000000
max,204.000000


In [20]:
sent_len.columns = ['sent_len', 'author']

all_features = word_counts.join(sent_len)

In [21]:
all_features

,responsible,previous,waiting,sixteen,naturally,little,teach,fact,bottle,correct,...,negative,impossible,artificial,sink,glad,liberality,text_sentence,text_source,sent_len,author
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,3,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll,6,Carroll
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll,126,Carroll
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(In, another, moment, down, went, Alice, after...",Carroll,23,Carroll
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(The, rabbit, -, hole, went, straight, on, lik...",Carroll,44,Carroll
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Either, the, well, was, very, deep, ,, or, sh...",Carroll,37,Carroll
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(First, ,, she, tried, to, look, down, and, ma...",Carroll,49,Carroll


In [22]:
for sent in alice_doc.sents:
    for token in sent:
        print([token.pos_, token.tag_, 'Carroll'])

['PROPN', 'NNP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['VERB', 'VBG', 'Carroll']
['PART', 'TO', 'Carroll']
['VERB', 'VB', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['ADP', 'IN', 'Carroll']
['VERB', 'VBG', 'Carroll']
['ADP', 'IN', 'Carroll']
['ADJ', 'PRP$', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['ADP', 'IN', 'Carroll']
['VERB', 'VBG', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PART', 'TO', 'Carroll']
['VERB', 'VB', 'Carroll']
['PUNCT', ':', 'Carroll']
['ADV', 'RB', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['ADV', 'RB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['VERB', 'VBN', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADJ', 'PRP$', 'Carroll']
['NOUN', 'NN', 'Carroll']
['VERB', 'VBD', 'Carroll']
['VERB', 'VBG', 'Carroll']
['PUNCT', ',', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['PRON'

['NOUN', 'NN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['PART', 'TO', 'Carroll']
['VERB', 'VB', 'Carroll']
['PART', 'RP', 'Carroll']
['ADV', 'RB', 'Carroll']
['PUNCT', '.', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', 'HYPH', 'Carroll']
['NOUN', 'NN', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['ADV', 'RB', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADV', 'RB', 'Carroll']
['PUNCT', ',', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADP', 'IN', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADV', 'RB', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PART', 'TO', 'Carroll']
['VERB', 'VB', 'Carroll']
['ADP', 'IN', 'Ca

['ADV', 'RB', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', '.', 'Carroll']
['VERB', 'VB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VB', 'Carroll']
['PUNCT', ':', 'Carroll']
['DET', 'DT', 'Carroll']
['VERB', 'MD', 'Carroll']
['VERB', 'VB', 'Carroll']
['NUM', 'CD', 'Carroll']
['NUM', 'CD', 'Carroll']
['NOUN', 'NNS', 'Carroll']
['PART', 'RP', 'Carroll']
['PUNCT', ',', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBP', 'Carroll']
['PUNCT', "''", 'Carroll']
['PUNCT', '-LRB-', 'Carroll']
['ADP', 'IN', 'Carroll']
['PUNCT', ',', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBP', 'Carroll']
['PUNCT', ',', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['VERB', 'VBN', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['NOUN', 'NNS', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['ADJ', '

['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'MD', 'Carroll']
['VERB', 'VB', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT', "''", 'Carroll']
['PUNCT', '-LRB-', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT', '-RRB-', 'Carroll']
['PUNCT', "''", 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBP', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'MD', 'Carroll']
['VERB', 'VB', 'Carroll']
['ADJ', 'PRP$', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', 'HYPH', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', '.', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['ADJ', 'PRP$', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', '.', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBP', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADV', 'RB',

['NOUN', 'NNS', 'Carroll']
['PUNCT', ',', 'Carroll']
['ADV', 'WRB', 'Carroll']
['ADV', 'RB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBZ', 'Carroll']
['VERB', 'VBG', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT', "''", 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['ADP', 'IN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['ADV', 'WRB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['DET', 'DT', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADJ', 'JJR', 'Carroll']
['PART', 'TO', 'Carroll']
['VERB', 'VB', 'Carroll']
['VERB', 'VBN', 'Carroll']
['PUNCT', ':', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['PRON', 'PRP', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['PUNCT', ',', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['NO

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




['PUNCT', ',', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['ADV', 'WRB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['PUNCT', ',', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'MD', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADV', 'RB', 'Carroll']
['VERB', 'VB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['PUNCT', ':', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'MD', 'Carroll']
['VERB', 'VB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['PRON', 'PRP', 'Carroll']
['ADJ', 'JJS', 'Carroll']
['PART', 'TO', 'Carroll']
['VERB', 'VB', 'Carroll']
['PART'

['VERB', 'VBZ', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT', "''", 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['DET', 'DT', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['CCONJ', 'CC', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADV', 'RB', 'Carroll']
['ADP', 'IN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['PUNCT', ',', 'Carroll']
['PUNCT', "''", 'Carroll']
['ADJ', 'WDT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', '.', 'Carroll']
['ADJ', 'WDT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT', "''", 'Carroll']
['PUNCT', ',', 'Carroll']
['VERB', 'VBG', 'Carroll']
['ADJ', 'PRP$', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['ADJ', 'PRP$', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PART', 'TO', 'Carroll']
['VERB', 'VB', 'Carroll']
['ADJ', 'WDT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB

['PRON', 'PRP', 'Carroll']
['DET', 'DT', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['NOUN', 'NN', 'Carroll']
['ADP', 'IN', 'Carroll']
['NOUN', 'NNS', 'Carroll']
['DET', 'DT', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT', "''", 'Carroll']
['CCONJ', 'CC', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['ADP', 'IN', 'Carroll']
['VERB', 'VBG', 'Carroll']
['ADP', 'IN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['ADV', 'WRB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'MD', 'Carroll']
['VERB', 'VB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT', "''", 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'MD', 'Carroll']
['VERB', 'VB', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['PUNCT', "''", 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['PUNCT', ':', 'Carroll']
['PUNCT', "''", 'Carroll']
['CCONJ', 'CC', 'Carroll']
['ADV', 'WRB', 'Carroll']
[

['VERB', 'VBD', 'Carroll']
['VERB', 'VBG', 'Carroll']
['PART', 'RP', 'Carroll']
['ADP', 'IN', 'Carroll']
['DET', 'DT', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', ',', 'Carroll']
['VERB', 'VBG', 'Carroll']
['ADP', 'IN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['ADP', 'IN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VBD', 'Carroll']
['PUNCT', ',', 'Carroll']
['PUNCT', "''", 'Carroll']
['INTJ', 'UH', 'Carroll']
['PUNCT', '.', 'Carroll']
['DET', 'DT', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['PUNCT', ',', 'Carroll']
['DET', 'DT', 'Carroll']
['PROPN', 'NNP', 'Carroll']
['PUNCT', '.', 'Carroll']
['INTJ', 'UH', 'Carroll']
['PUNCT', '.', 'Carroll']
['VERB', 'MD', 'Carroll']
['ADV', 'RB', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VB', 'Carroll']
['ADJ', 'JJ', 'Carroll']
['ADP', 'IN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['VERB', 'VB', 'Carroll']
['VERB', 'VBN', 'Carroll']
['PRON', 'PRP', 'Carroll']
['NOUN', 'NN', 'Carroll']
['PUNCT', '.', 'Carroll']
['PUNCT

In [23]:
list(sentences.columns)

[0, 1]

In [24]:
sentences.columns = ['sent_text', 'author']
list(sentences.columns)

['sent_text', 'author']